In this notebook, we’ll use `featuretools` to automatically generate new features across all datasets, making feature engineering both faster and more powerful. `Featuretools` is an open-source Python library designed specifically to create valuable features from raw data for machine learning, streamlining the entire process so we can skip manual feature creation. By using mathematical transformations (like summing or averaging) and connecting related datasets, `featuretools` helps us build rich, informative features effortlessly—even in complex, multi-table datasets like ours.

One of the standout features of `featuretools` is its Deep Feature Synthesis (DFS) technique, which stacks multiple transformations to create highly informative "deep" features. For instance, it can automatically create a feature like "average transaction amount per month" by combining simpler data points. This makes our dataset not only more insightful but also better prepared for accurate model training. By letting `featuretools` handle the heavy lifting, we can focus on making our model more effective and our workflow more efficient!

In [2]:
import pandas as pd
import numpy as np

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

# import the functions from the file
from notebook_setup import RootPath, config
from my_functions import pie_plot

# Automatically Reload Changes if there are frequent changes to my xx.py file
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

# Automatically Reload Changes if there are frequent changes to my xx.py file
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from featuretools.selection import (
    remove_highly_correlated_features,
    remove_highly_null_features,
    remove_single_value_features,
)

# import the functions from the file
from my_functions import (
    convert_to_categorical, 
    encode_and_one_hot, 
    summarize_dataframe,
    get_top_correlations, 
    missing_values_table, 
    preprocess_and_reduce_features,
    merge_data_on_index
)

In [5]:
# Load all sampling data and drop "Unnamed" which is the index column
app_df = pd.read_csv(f'{config.CleanDataPath}app_df.csv')
app_df = app_df.loc[:, ~app_df.columns.str.contains('^Unnamed')]

bureau_df = pd.read_csv(f'{config.CleanDataPath}bureau_df.csv')
bureau_df = bureau_df.loc[:, ~bureau_df.columns.str.contains('^Unnamed')]

bureau_balance_df = pd.read_csv(f'{config.CleanDataPath}bureau_balance_df.csv')
bureau_balance_df = bureau_balance_df.loc[:, ~bureau_balance_df.columns.str.contains('^Unnamed')]

previous_df = pd.read_csv(f'{config.CleanDataPath}previous_df.csv')
previous_df = previous_df.loc[:, ~previous_df.columns.str.contains('^Unnamed')]

cash_df = pd.read_csv(f'{config.CleanDataPath}cash_df.csv')
cash_df = cash_df.loc[:, ~cash_df.columns.str.contains('^Unnamed')]

instalments_df = pd.read_csv(f'{config.CleanDataPath}instalments_df.csv')
instalments_df = instalments_df.loc[:, ~instalments_df.columns.str.contains('^Unnamed')]

credit_df = pd.read_csv(f'{config.CleanDataPath}credit_df.csv')
credit_df = credit_df.loc[:, ~credit_df.columns.str.contains('^Unnamed')]
credit_df.head(5)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2740914,340339,-1,1.310354,0.500,10800.0,10800.00,0.0,0.00,0.888960,...,1.276882,1.276635,2.0,2.0,0.0,0.0,6.0,Active,0.0,0.0
1,1864742,303581,-1,0.000000,-0.500,0.0,0.00,0.0,0.00,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.0,59.0,Active,0.0,0.0
2,1189095,231608,-4,4.505954,1.750,9000.0,9000.00,0.0,0.00,3.395033,...,4.507100,4.506230,1.0,1.0,0.0,0.0,22.0,Active,0.0,0.0
3,1683604,217113,-1,0.683794,-0.375,0.0,4412.25,0.0,4412.25,0.491147,...,0.686007,0.685874,0.0,4.0,0.0,4.0,6.0,Active,0.0,0.0
4,2252971,193247,-4,0.233098,2.250,0.0,0.00,0.0,0.00,0.333333,...,0.233095,0.233050,0.0,0.0,0.0,0.0,6.0,Active,0.0,0.0


### Automated Feature Engineering with Featuretools
To keep things straightforward, let’s go with the default options for automated feature engineering. These settings are designed to quickly generate useful features based on data types and relationships, helping to boost model performance without the need for complex, hands-on engineering. We’ll set the feature depth to a maximum of 2 levels, which is usually a great balance between capturing valuable information and keeping things efficient.

Once the process is complete, we’ll fine-tune the feature set by removing any that meet the following criteria:

1. Features with a high percentage of missing values (threshold: 90%)
2. Single-value features (as long as they don’t contain NaN values)
3. Highly correlated features (threshold: 90%)

This streamlined approach will let us retain the most impactful features, enhancing our model’s effectiveness and simplicity.

In [7]:
# Create the EntitySet with sampled data
es = ft.EntitySet(id="customer_data")
es = es.add_dataframe(dataframe_name='app', dataframe=app_df, index='SK_ID_CURR')
es = es.add_dataframe(dataframe_name='bureau', dataframe=bureau_df, index='SK_ID_BUREAU')
es = es.add_dataframe(dataframe_name='previous', dataframe=previous_df, index='SK_ID_PREV')
es = es.add_dataframe(dataframe_name='bureau_balance', dataframe=bureau_balance_df, make_index=True, index='bureaubalance_index')
es = es.add_dataframe(dataframe_name='cash', dataframe=cash_df, make_index=True, index='cash_index')
es = es.add_dataframe(dataframe_name='instalments', dataframe=instalments_df, make_index=True, index='instalments_index')
es = es.add_dataframe(dataframe_name='credit', dataframe=credit_df, make_index=True, index='credit_index')

# Define relationships in the EntitySet
es = es.add_relationship('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR')
es = es.add_relationship('app', 'SK_ID_CURR', 'previous', 'SK_ID_CURR')
es = es.add_relationship('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU')
es = es.add_relationship('previous', 'SK_ID_PREV', 'cash', 'SK_ID_PREV')
es = es.add_relationship('previous', 'SK_ID_PREV', 'instalments', 'SK_ID_PREV')
es = es.add_relationship('previous', 'SK_ID_PREV', 'credit', 'SK_ID_PREV')


In [8]:
# Deep Feature Synthesis (DFS) with default primitives
fm, features = ft.dfs(entityset = es, target_dataframe_name= 'app',
                      max_depth = 2, features_only=False, verbose = True)

print(fm.shape)
fm.head(10)

Built 2088 features
Elapsed: 36:48 | Progress: 100%|██████████
(30751, 2088)


,TARGET,COUNT(bureau),MAX(bureau.AMT_ANNUITY),MAX(bureau.AMT_CREDIT_MAX_OVERDUE),MAX(bureau.AMT_CREDIT_SUM),MAX(bureau.AMT_CREDIT_SUM_DEBT),MAX(bureau.AMT_CREDIT_SUM_LIMIT),MAX(bureau.AMT_CREDIT_SUM_OVERDUE),MAX(bureau.CNT_CREDIT_PROLONG),MAX(bureau.CREDIT_DAY_OVERDUE),...,SUM(credit.previous.DAYS_LAST_DUE),SUM(credit.previous.DAYS_LAST_DUE_1ST_VERSION),SUM(credit.previous.DAYS_TERMINATION),SUM(credit.previous.HOUR_APPR_PROCESS_START),SUM(credit.previous.NFLAG_INSURED_ON_APPROVAL),SUM(credit.previous.NFLAG_LAST_APPL_IN_DAY),SUM(credit.previous.RATE_DOWN_PAYMENT),SUM(credit.previous.RATE_INTEREST_PRIMARY),SUM(credit.previous.RATE_INTEREST_PRIVILEGED),SUM(credit.previous.SELLERPLACE_AREA)
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
307474,0,5,NaN,0.000,3.116422,0.000000,0.000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
412537,0,4,0.033333,0.000,0.317765,0.639861,176852.565,0.0,2.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
149084,0,1,NaN,NaN,0.588603,0.000000,0.000,0.0,0.0,0.0,...,11495.037873,10354.627451,11634.342577,702.0,0.0,0.0,0.0,0.0,0.0,-1.835294
364692,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
155423,0,5,0.000000,NaN,1.072048,2.653891,0.000,0.0,0.0,0.0,...,15.592264,22833.281046,25655.216966,1118.0,0.0,0.0,0.0,0.0,0.0,2401.929412
165164,0,12,18.730333,0.000,2.755305,12.389950,3068.775,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
427814,0,6,NaN,0.000,2.321964,17.055449,0.000,0.0,1.0,0.0,...,2.014504,2655.032680,2983.164763,110.0,0.0,0.0,0.0,0.0,0.0,93.764706
248940,0,5,NaN,25803.945,9.435970,33.960908,0.000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
218738,0,12,NaN,11493.765,1.762233,5.257485,0.000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [9]:
features[-20:]

[<Feature: STD(credit.previous.SELLERPLACE_AREA)>,
 <Feature: SUM(credit.previous.AMT_ANNUITY)>,
 <Feature: SUM(credit.previous.AMT_APPLICATION)>,
 <Feature: SUM(credit.previous.AMT_CREDIT)>,
 <Feature: SUM(credit.previous.AMT_DOWN_PAYMENT)>,
 <Feature: SUM(credit.previous.AMT_GOODS_PRICE)>,
 <Feature: SUM(credit.previous.CNT_PAYMENT)>,
 <Feature: SUM(credit.previous.DAYS_DECISION)>,
 <Feature: SUM(credit.previous.DAYS_FIRST_DRAWING)>,
 <Feature: SUM(credit.previous.DAYS_FIRST_DUE)>,
 <Feature: SUM(credit.previous.DAYS_LAST_DUE)>,
 <Feature: SUM(credit.previous.DAYS_LAST_DUE_1ST_VERSION)>,
 <Feature: SUM(credit.previous.DAYS_TERMINATION)>,
 <Feature: SUM(credit.previous.HOUR_APPR_PROCESS_START)>,
 <Feature: SUM(credit.previous.NFLAG_INSURED_ON_APPROVAL)>,
 <Feature: SUM(credit.previous.NFLAG_LAST_APPL_IN_DAY)>,
 <Feature: SUM(credit.previous.RATE_DOWN_PAYMENT)>,
 <Feature: SUM(credit.previous.RATE_INTEREST_PRIMARY)>,
 <Feature: SUM(credit.previous.RATE_INTEREST_PRIVILEGED)>,
 <Feature:

#### Remove Highly Null Features

In [11]:
fm2 =  remove_highly_null_features(fm, pct_null_threshold=0.90)
print(fm2.shape)
#fm2.head(5)

(30751, 1867)


#### Remove Single Value Features

In [13]:
fm3, features3 = remove_single_value_features(
    fm2, features=features, count_nan_as_value=True
)
print(fm3.shape)
#fm3.head()

(30751, 1851)


#### Remove Highly Correlated Features

In [15]:
fm4, features4 = remove_highly_correlated_features(
    fm3, features=features3, pct_corr_threshold=0.9)


In [16]:
pd.options.display.max_columns = 2000
print(fm4.shape)
fm4.head()

(30751, 774)


,TARGET,COUNT(bureau),MAX(bureau.AMT_ANNUITY),MAX(bureau.AMT_CREDIT_MAX_OVERDUE),MAX(bureau.AMT_CREDIT_SUM),MAX(bureau.AMT_CREDIT_SUM_DEBT),MAX(bureau.AMT_CREDIT_SUM_LIMIT),MAX(bureau.AMT_CREDIT_SUM_OVERDUE),MAX(bureau.CNT_CREDIT_PROLONG),MAX(bureau.CREDIT_DAY_OVERDUE),MAX(bureau.DAYS_CREDIT),MAX(bureau.DAYS_CREDIT_ENDDATE),MAX(bureau.DAYS_CREDIT_UPDATE),MAX(bureau.DAYS_ENDDATE_FACT),MEAN(bureau.AMT_CREDIT_MAX_OVERDUE),MEAN(bureau.AMT_CREDIT_SUM),MEAN(bureau.AMT_CREDIT_SUM_DEBT),MEAN(bureau.AMT_CREDIT_SUM_LIMIT),MEAN(bureau.CNT_CREDIT_PROLONG),MEAN(bureau.CREDIT_DAY_OVERDUE),MEAN(bureau.DAYS_CREDIT),MEAN(bureau.DAYS_CREDIT_ENDDATE),MEAN(bureau.DAYS_CREDIT_UPDATE),MEAN(bureau.DAYS_ENDDATE_FACT),MIN(bureau.AMT_ANNUITY),MIN(bureau.AMT_CREDIT_MAX_OVERDUE),MIN(bureau.AMT_CREDIT_SUM),MIN(bureau.AMT_CREDIT_SUM_DEBT),MIN(bureau.AMT_CREDIT_SUM_LIMIT),MIN(bureau.AMT_CREDIT_SUM_OVERDUE),MIN(bureau.CNT_CREDIT_PROLONG),MIN(bureau.CREDIT_DAY_OVERDUE),MIN(bureau.DAYS_CREDIT),MIN(bureau.DAYS_CREDIT_ENDDATE),MIN(bureau.DAYS_CREDIT_UPDATE),MIN(bureau.DAYS_ENDDATE_FACT),MODE(bureau.CREDIT_ACTIVE),MODE(bureau.CREDIT_CURRENCY),MODE(bureau.CREDIT_TYPE),NUM_UNIQUE(bureau.CREDIT_ACTIVE),NUM_UNIQUE(bureau.CREDIT_CURRENCY),NUM_UNIQUE(bureau.CREDIT_TYPE),SKEW(bureau.AMT_ANNUITY),SKEW(bureau.AMT_CREDIT_MAX_OVERDUE),SKEW(bureau.AMT_CREDIT_SUM),SKEW(bureau.AMT_CREDIT_SUM_DEBT),SKEW(bureau.AMT_CREDIT_SUM_LIMIT),SKEW(bureau.AMT_CREDIT_SUM_OVERDUE),SKEW(bureau.CNT_CREDIT_PROLONG),SKEW(bureau.DAYS_CREDIT),SKEW(bureau.DAYS_CREDIT_ENDDATE),SKEW(bureau.DAYS_CREDIT_UPDATE),SKEW(bureau.DAYS_ENDDATE_FACT),STD(bureau.DAYS_CREDIT),STD(bureau.DAYS_ENDDATE_FACT),SUM(bureau.AMT_CREDIT_SUM),SUM(bureau.DAYS_CREDIT_ENDDATE),SUM(bureau.DAYS_CREDIT_UPDATE),COUNT(bureau_balance),MAX(bureau_balance.MONTHS_BALANCE),MEAN(bureau_balance.MONTHS_BALANCE),MODE(bureau_balance.STATUS),NUM_UNIQUE(bureau_balance.STATUS),SKEW(bureau_balance.MONTHS_BALANCE),COUNT(previous),MAX(previous.AMT_ANNUITY),MAX(previous.AMT_APPLICATION),MAX(previous.AMT_DOWN_PAYMENT),MAX(previous.CNT_PAYMENT),MAX(previous.DAYS_DECISION),MAX(previous.DAYS_FIRST_DRAWING),MAX(previous.DAYS_FIRST_DUE),MAX(previous.DAYS_LAST_DUE),MAX(previous.DAYS_LAST_DUE_1ST_VERSION),MAX(previous.HOUR_APPR_PROCESS_START),MAX(previous.NFLAG_INSURED_ON_APPROVAL),MAX(previous.NFLAG_LAST_APPL_IN_DAY),MAX(previous.RATE_DOWN_PAYMENT),MAX(previous.SELLERPLACE_AREA),MEAN(previous.AMT_ANNUITY),MEAN(previous.AMT_APPLICATION),MEAN(previous.AMT_DOWN_PAYMENT),MEAN(previous.CNT_PAYMENT),MEAN(previous.DAYS_DECISION),MEAN(previous.DAYS_FIRST_DRAWING),MEAN(previous.DAYS_FIRST_DUE),MEAN(previous.DAYS_LAST_DUE),MEAN(previous.DAYS_LAST_DUE_1ST_VERSION),MEAN(previous.HOUR_APPR_PROCESS_START),MEAN(previous.NFLAG_INSURED_ON_APPROVAL),MEAN(previous.NFLAG_LAST_APPL_IN_DAY),MEAN(previous.RATE_DOWN_PAYMENT),MEAN(previous.SELLERPLACE_AREA),MIN(previous.AMT_ANNUITY),MIN(previous.AMT_APPLICATION),MIN(previous.AMT_DOWN_PAYMENT),MIN(previous.AMT_GOODS_PRICE),MIN(previous.CNT_PAYMENT),MIN(previous.DAYS_DECISION),MIN(previous.DAYS_FIRST_DRAWING),MIN(previous.DAYS_FIRST_DUE),MIN(previous.DAYS_LAST_DUE),MIN(previous.HOUR_APPR_PROCESS_START),MIN(previous.NFLAG_INSURED_ON_APPROVAL),MIN(previous.NFLAG_LAST_APPL_IN_DAY),MIN(previous.RATE_DOWN_PAYMENT),MIN(previous.SELLERPLACE_AREA),MODE(previous.CHANNEL_TYPE),MODE(previous.CODE_REJECT_REASON),MODE(previous.NAME_CASH_LOAN_PURPOSE),MODE(previous.NAME_CLIENT_TYPE),MODE(previous.NAME_CONTRACT_STATUS),MODE(previous.NAME_CONTRACT_TYPE),MODE(previous.NAME_GOODS_CATEGORY),MODE(previous.NAME_PAYMENT_TYPE),MODE(previous.NAME_PORTFOLIO),MODE(previous.NAME_PRODUCT_TYPE),MODE(previous.NAME_SELLER_INDUSTRY),MODE(previous.NAME_TYPE_SUITE),MODE(previous.NAME_YIELD_GROUP),MODE(previous.PRODUCT_COMBINATION),MODE(previous.WEEKDAY_APPR_PROCESS_START),NUM_UNIQUE(previous.CHANNEL_TYPE),NUM_UNIQUE(previous.CODE_REJECT_REASON),NUM_UNIQUE(previous.NAME_CASH_LOAN_PURPOSE),NUM_UNIQUE(previous.NAME_CLIENT_TYPE),NUM_UNIQUE(previous.NAME_CONTRACT_STATUS

In [17]:
summarize_dataframe(fm4)


Data Total Number of Each Type:
 float64     487
Int64       209
int64         7
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      4
category      3
category      2
category      2
category      2
category      2
category      2
category      2
Name: count, dtype: int64

Float Types Count:
 MAX(bureau.AMT_ANNUITY)                   4346
MAX(bureau.AMT_CREDIT_MAX_OVERDUE)        5847
MAX(bureau.AMT_CREDIT_SUM)               10014
MAX(bureau.AMT_CREDIT_SUM_DEBT)          17748
MAX(bureau.AMT_CREDIT_SUM_LIMIT)          3844
                                         ...  
SUM(credit.previous.AMT_GOODS_PRICE)       587
SUM(credit.previous.DAYS_FIRST_DUE)       3641
SUM(credit.previous.DAYS_LAST_DUE)        2082
SUM(credit.previous.DAYS_TERMINATION)     1102
SUM(credit.previous.SELLERPLACE_AREA)     2050
Length: 487, dtype

In [18]:
columns_to_check = ['SK_ID_BUREAU', 'SK_ID_CURR', 'SK_ID_PREV', 'TARGET']

# Check if each column exists in the DataFrame and print the result
for column in columns_to_check:
    if column in fm4.columns:
        print(f"Column '{column}' is in the DataFrame.")
    else:
        print(f"Column '{column}' is NOT in the DataFrame.")

Column 'SK_ID_BUREAU' is NOT in the DataFrame.
Column 'SK_ID_CURR' is NOT in the DataFrame.
Column 'SK_ID_PREV' is NOT in the DataFrame.
Column 'TARGET' is in the DataFrame.


In [19]:
fm5 = fm4.copy()
target = fm5['TARGET']
fm5 = fm5.drop(columns = 'TARGET')
# convert 'object' to categorical features
fm5 = convert_to_categorical(fm5, min_unique=2, max_unique=20)

Converted columns to categorical: []


In [20]:
# perform one-hot encoding for categorical features
fm5 = encode_and_one_hot(fm5)

DataFrame shape before encoding: (30751, 773)
0 columns were label encoded.
DataFrame shape after encoding: (30751, 1258)


In [26]:
# load df3, df4, df3_new_features, and df4_new_features
# load the data
df3 = pd.read_csv(f'{config.CleanDataPath}df3.csv', index_col='SK_ID_CURR')
df3 = df3.loc[:, ~df3.columns.str.contains('^Unnamed')]
df3_new_features = pd.read_csv(f'{config.CleanDataPath}df3_new_features.csv', index_col='SK_ID_CURR')
df3_new_features = df3_new_features.loc[:, ~df3_new_features.columns.str.contains('^Unnamed')]

df4 = pd.read_csv(f'{config.CleanDataPath}df4.csv', index_col='SK_ID_CURR')
df4 = df4.loc[:, ~df4.columns.str.contains('^Unnamed')]
df4_new_features = pd.read_csv(f'{config.CleanDataPath}df4_new_features.csv', index_col='SK_ID_CURR')
df4_new_features = df4_new_features.loc[:, ~df4_new_features.columns.str.contains('^Unnamed')]


In [28]:
# Merge data together
fm5_df4 = merge_data_on_index(fm5, df4, how='inner')
fm5_df4_new_features = merge_data_on_index(fm5, df4_new_features, how='inner')

First data input shape: (30751, 1258)
Second data input shape: (30751, 161)
Combined data shape: (30751, 1419)
First data input shape: (30751, 1258)
Second data input shape: (30751, 214)
Combined data shape: (30751, 1472)


To boost computational efficiency and reduce noise, we’ll use Factor Analysis of Mixed Data (FAMD) to streamline the dataset by reducing its dimensionality. We’ll retain components until we capture 70% of the cumulative variance for model experimentation. By setting this threshold, we create a dataset that’s optimized for effective model testing and analysis while maintaining the most informative features.

FAMD is a great choice here because it’s built to handle both numerical and categorical variables, making it perfect for mixed datasets like ours. By reducing the dimensionality, FAMD retains the most critical variance, helping us filter out redundant or noisy features and resulting in a more efficient and manageable dataset for model training.

As with any dimensionality reduction method, FAMD does come with trade-offs. With our 70% variance threshold, we’ll lose about 30% of the variance, which means some information may be sacrificed. Additionally, the components created by FAMD are combinations of multiple original features, which can make them a bit harder to interpret. Nonetheless, the benefits of a streamlined, computationally efficient dataset often outweigh these minor drawbacks, especially when it comes to preparing our data for effective modeling!


In [43]:
import time

# Record start time
start_time = time.time()
reduced_df = preprocess_and_reduce_features(fm5_df4, n_components=120, target_variance=0.70) 
# Record end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


Number of Components: 120
Cumulative Variance: 0.6273510224734017
Number of Components: 121
Cumulative Variance: 0.6296745975252099
Number of Components: 122
Cumulative Variance: 0.6319613449832183
Number of Components: 123
Cumulative Variance: 0.6340994295835857
Number of Components: 124
Cumulative Variance: 0.6359731590012577
Number of Components: 125
Cumulative Variance: 0.6387124750466693
Number of Components: 126
Cumulative Variance: 0.6411434534318146
Number of Components: 127
Cumulative Variance: 0.6431347808238193
Number of Components: 128
Cumulative Variance: 0.6452237970355311
Number of Components: 129
Cumulative Variance: 0.6475538249060243
Number of Components: 130
Cumulative Variance: 0.649654452602891
Number of Components: 131
Cumulative Variance: 0.6519035047971888
Number of Components: 132
Cumulative Variance: 0.6537936668121904
Number of Components: 133
Cumulative Variance: 0.6560990941880118
Number of Components: 134
Cumulative Variance: 0.658379114423537
Number of Co

In [44]:

start_time = time.time()
reduced_df_new_features = preprocess_and_reduce_features(fm5_df4_new_features, n_components=120, target_variance=0.70)
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


Number of Components: 120
Cumulative Variance: 0.6161041029129454
Number of Components: 121
Cumulative Variance: 0.6179054491771163
Number of Components: 122
Cumulative Variance: 0.6210253134796486
Number of Components: 123
Cumulative Variance: 0.6228331021986655
Number of Components: 124
Cumulative Variance: 0.6249560165683875
Number of Components: 125
Cumulative Variance: 0.6271415961498457
Number of Components: 126
Cumulative Variance: 0.6297655022767376
Number of Components: 127
Cumulative Variance: 0.631906429756533
Number of Components: 128
Cumulative Variance: 0.6337998708016509
Number of Components: 129
Cumulative Variance: 0.6359125101222657
Number of Components: 130
Cumulative Variance: 0.6383061166713427
Number of Components: 131
Cumulative Variance: 0.640682410910005
Number of Components: 132
Cumulative Variance: 0.6423867533169529
Number of Components: 133
Cumulative Variance: 0.6445124239891681
Number of Components: 134
Cumulative Variance: 0.6467248308149817
Number of Co

In [49]:
#merge again with 60 top corr data
df3_reduced_df = merge_data_on_index(df3, reduced_df, how='inner')
df3_reduced_df_new_features = merge_data_on_index(df3_new_features, reduced_df_new_features, how='inner')

First data input shape: (30751, 60)
Second data input shape: (30751, 154)
Combined data shape: (30751, 214)
First data input shape: (30751, 60)
Second data input shape: (30751, 160)
Combined data shape: (30751, 220)


In [51]:
# Saving the data for model implementation
df3_reduced_df.to_csv(f'{config.CleanDataPath}df3_reduced_df.csv', index=True, index_label='SK_ID_CURR')
df3_reduced_df_new_features.to_csv(f'{config.CleanDataPath}df3_reduced_df_new_features.csv', index=True, index_label='SK_ID_CURR')
